In [1]:
import os
import csv
import math
from collections import defaultdict

##############################################
# User Defined Variables
##############################################
ActiveDate = '20230505' # Format: YYYYMMDD
FullFileName = 'C:/Users/DSMIADAK/OneDrive - Intel Corporation/76 Fleet/20230505/X Edge/Defects_0505_01_32_07.tab'
#Input file dependent, what is in the first column?
#If Process Equipment, set to True
ProcessEquipFirst = True
##############################################

Output_File_1 = os.path.dirname(FullFileName) + '/output_file1_' + ActiveDate + '.csv'
Output_File_2 = os.path.dirname(FullFileName) + '/output_file2_' + ActiveDate + '.csv'

# Open the tab delimited input file
with open(FullFileName, 'r') as tsv_file:
    # Create a reader object for the input file
    tsv_reader = csv.reader(tsv_file, delimiter='\t')
    
    # Create a list to hold the rows we want to keep
    rows_to_keep = []
    #rows_to_keep.append('Inspection Date', 'Lot', 'Slot', 'WaferX', 'WaferY', 'WaferR', 'Location', 'Zone')
    next(tsv_reader)
    # Loop through each row in the input file
    for row in tsv_reader:
        if ProcessEquipFirst:
            #If Process Equipment is first column of DefectMapper output file ~11/22 to Present files
            InspectionDate = row[3]
            Lot = row[5]
            Slot = row[6]
            ProcessEquip = row[0]
            LinkStartTime = row[13]
            WaferX = int(row[24])
            WaferY = int(row[25])
            WaferR = float(row[27])
        else:
            # If Inspection Time is first column of DefectMapper output file ~ Older to 10/22 files
            InspectionDate = row[1]
            Lot = row[3]
            Slot = row[4]
            ProcessEquip = row[10]
            LinkStartTime = row[12]
            WaferX = int(row[24])
            WaferY = int(row[25])
            WaferR = float(row[27])
        Zone = ''
        if WaferX != 0 and WaferY != 0:
            Angle = math.degrees(2*math.atan(WaferY/(WaferX+math.sqrt(WaferX**2+WaferY**2))))
             
        #Determine if defect is at the extreme edge and in a non-film pulling region.
        if WaferR > 135000000:
            if WaferR >= 145000000:
                Location = "Extreme Edge Count"
                if 0 <= Angle <= 55 or 62 <= Angle <= 110 or 130 <= Angle <= 147 or 152 <= Angle <= 180:
                    Zone = 'Extreme Edge Count - Non-Film Pulling Region'
                elif -180 < Angle <= -152 or -132 <= Angle <= -110 or -102 <= Angle <= -63 or -45 <= Angle <= -30 or -13 <= Angle <= 0:
                    Zone = 'Extreme Edge Count - Non-Film Pulling Region'
                else:
                    Zone = 'Extreme Edge Count - Film Pulling Region'
            else:
                Location = "Edge Count"
                if 0 <= Angle <= 55 or 62 <= Angle <= 110 or 130 <= Angle <= 147 or 152 <= Angle <= 180:
                    Zone = 'Edge Count - Non-Film Pulling Region'
                elif -180 < Angle <= -152 or -132 <= Angle <= -110 or -102 <= Angle <= -63 or -45 <= Angle <= -30 or -13 <= Angle <= 0:
                    Zone = 'Edge Count - Non-Film Pulling Region'
                else:
                    Zone = 'Edge Count - Film Pulling Region'
        else:
            Location = "Center Count"
            Zone = 'Center Count'
        # Extract columns and add them to the list 
        if ProcessEquipFirst:      
            # Process Equipment First Column
            rows_to_keep.append([row[0], row[3], row[13], row[5], row[6], row[24], row[25], row[27], Angle, Location, Zone])
        else:
            # Inspection Time First Column
            rows_to_keep.append([ProcessEquip, InspectionDate, LinkStartTime, Lot, Slot, row[24], row[25], row[27], Angle, Location, Zone])
    # Open the output CSV file
    with open(Output_File_1, 'w', newline='') as csv_file:
        # Create a writer object for the output file
        csv_writer = csv.writer(csv_file)
        header = ['Process Equipment', 'Inspection Date', 'Link Start Time', 'Lot', 'Slot', 'WaferX', 'WaferY', 'WaferR', 'Angle', 'Location', 'Zone']
        csv_writer.writerow(header)
        # Write the rows we want to keep to the output file
        csv_writer.writerows(rows_to_keep)

print("Output File 1 Generated")

# Open the CSV file and create a dictionary to store the counts for each row
with open(Output_File_1, 'r') as csv_file:

    reader = csv.reader(csv_file)
    next(reader)  # skip the header row
    counts = defaultdict(int)
    for row in reader:
        key = (row[0], row[1], row[2], row[3], row[4], row[9], row[10])
        counts[key] += 1

# Output the unique rows and their counts to a new CSV file
with open(Output_File_2, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Process Equipment', 'Inspection Date', 'Link Start Time', 'Lot', 'Slot', 'Location', 'Zone', 'Count'])
    for key, count in counts.items():
        writer.writerow([key[0], key[1], key[2], key[3], key[4], key[5], key[6], count])
        
print("Output File 2 Generated")

Output File 1 Generated
Output File 2 Generated
